## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [76]:
%%sql
postgresql:///animal_shelters

'Connected: @animal_shelters'

## We will first take a look at all of columns and their data types

#### Table: animals

In [77]:
%%sql

SELECT
    column_name,
    data_type
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    TABLE_NAME = 'animals'


 * postgresql:///animal_shelters
6 rows affected.


column_name,data_type
animalid,character varying
birthdate,character varying
animaltype,character varying
color,character varying
weight,numeric
location,character varying


#### Table: sponsored_animals

In [78]:
%%sql


SELECT
    column_name,
    data_type
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    TABLE_NAME = 'sponsored_animals'

 * postgresql:///animal_shelters
3 rows affected.


column_name,data_type
sponsorid,character varying
animaltype,character varying
location,character varying


#### Table: location_costs

In [79]:
%%sql


SELECT
    column_name,
    data_type
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    TABLE_NAME = 'location_costs'

 * postgresql:///animal_shelters
2 rows affected.


column_name,data_type
location,character varying
costs,integer


#### Table: age_costs

In [80]:
%%sql


SELECT
    column_name,
    data_type
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    TABLE_NAME = 'age_costs'

 * postgresql:///animal_shelters
2 rows affected.


column_name,data_type
age,character varying
costs,integer


#### Table: size_costs

In [81]:
%%sql


SELECT
    column_name,
    data_type
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    TABLE_NAME = 'size_costs'

 * postgresql:///animal_shelters
4 rows affected.


column_name,data_type
sizeid,character varying
animaltype,character varying
size,character varying
costs,integer


# The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the size_costs table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:
 
	           Small 	         Medium 	        Large
    Dog 	<= 10 lbs 	10 lbs < and <= 30 lbs 	30 lbs <
    Cat 	<= 5 lbs 	5 lbs < and <= 7 lbs 	7 lbs <
    Bird 	<= 0.7 lbs 	0.7 lbs < and <= 1.1 lbs 	1.1 lbs <



### Create bins per the new size classifications

In [82]:
%%sql

-- We use CASE to classify the animals

SELECT
animaltype,
weight,
location,
CASE
    WHEN animaltype = 'Bird' AND weight <= 0.7     
        OR animaltype = 'Cat' AND weight <= 5 
        OR animaltype = 'Dog' AND weight <= 10 THEN 'Small'
    WHEN animaltype = 'Bird' AND weight > 1.1 
        OR animaltype = 'Cat' AND weight > 7 
        OR animaltype = 'Dog' AND weight > 30 THEN 'Large'
    ELSE 'Medium'
END AS size
FROM animals
LIMIT 5;

 * postgresql:///animal_shelters
5 rows affected.


animaltype,weight,location,size
Cat,4.0,Fort Worth,Small
Cat,7.0,Houston,Medium
Dog,45.0,Austin,Large
Cat,3.0,Houston,Small
Cat,9.0,Houston,Large


# 2. Older animals cost more, and so an age cost (contained in the age_costs table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).

### Create an age column

In [83]:
%%sql

SELECT
    animaltype,
    weight,
    location,
    2021 - EXTRACT(YEAR FROM CAST(birthdate AS DATE)) as age
FROM animals
LIMIT 5;

 * postgresql:///animal_shelters
5 rows affected.


animaltype,weight,location,age
Cat,4.0,Fort Worth,2.0
Cat,7.0,Houston,9.0
Dog,45.0,Austin,8.0
Cat,3.0,Houston,6.0
Cat,9.0,Houston,7.0


# 3. There is a location cost depending on where the animal is sheltered (contained in the location_costs table).

There is no need to change anything here. The animals table already contains a location column. This
column can be used later for joining if needed.

# 4. The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the sponsored_animals table).

### Next we can find the animals that do not have sponsors

In [84]:
%%sql


SELECT
    animaltype,
    weight,
    location
FROM animals
WHERE animalid NOT IN (SELECT sponsorid
                      FROM sponsored_animals
                      )
LIMIT 5;

 * postgresql:///animal_shelters
5 rows affected.


animaltype,weight,location
Cat,7.0,Houston
Dog,45.0,Austin
Cat,9.0,Houston
Dog,39.0,Austin
Dog,3.0,Fort Worth


Great! Now that we have figured out how to select everything we need for te final table, we can start be creating a new table containg all the queries that were just created!

# Creating new table

In [85]:
%%sql

CREATE TABLE animals_updated AS

SELECT 
    animaltype,
    weight,
    location,
    -- Enter the age query here.
    2021 - EXTRACT(YEAR FROM CAST(birthdate AS DATE)) as age,
    -- Enter the case query here.
       CASE
    WHEN animaltype = 'Bird' AND weight <= 0.7     
        OR animaltype = 'Cat' AND weight <= 5 
        OR animaltype = 'Dog' AND weight <= 10 THEN 'Small'
    WHEN animaltype = 'Bird' AND weight > 1.1 
        OR animaltype = 'Cat' AND weight > 7 
        OR animaltype = 'Dog' AND weight > 30 THEN 'Large'
    ELSE 'Medium'
    END AS size
FROM animals
-- Now we filter by the animals not sponsored
WHERE 
    animalid NOT IN (SELECT 
                        sponsorid
                    FROM sponsored_animals);


 * postgresql:///animal_shelters
(psycopg2.errors.DuplicateTable) relation "animals_updated" already exists

[SQL: CREATE TABLE animals_updated AS

SELECT 
    animaltype,
    weight,
    location,
    -- Enter the age query here.
    2021 - EXTRACT(YEAR FROM CAST(birthdate AS DATE)) as age,
    -- Enter the case query here.
       CASE
    WHEN animaltype = 'Bird' AND weight <= 0.7     
        OR animaltype = 'Cat' AND weight <= 5 
        OR animaltype = 'Dog' AND weight <= 10 THEN 'Small'
    WHEN animaltype = 'Bird' AND weight > 1.1 
        OR animaltype = 'Cat' AND weight > 7 
        OR animaltype = 'Dog' AND weight > 30 THEN 'Large'
    ELSE 'Medium'
    END AS size
FROM animals
-- Now we filter by the animals not sponsored
WHERE 
    animalid NOT IN (SELECT 
                        sponsorid
                    FROM sponsored_animals);]
(Background on this error at: http://sqlalche.me/e/14/f405)


### Now we can check to make sure the table exists 

In [86]:
%%sql

SELECT 
    *
FROM animals_updated
LIMIT 10;

 * postgresql:///animal_shelters
10 rows affected.


animaltype,weight,location,age,size
Cat,7.0,Houston,9,Medium
Dog,45.0,Austin,8,Large
Cat,9.0,Houston,7,Large
Dog,39.0,Austin,4,Large
Dog,3.0,Fort Worth,5,Small
Dog,24.0,Fort Worth,3,Medium
Dog,16.0,Austin,4,Medium
Dog,13.0,Houston,8,Medium
Cat,9.0,Houston,4,Large
Cat,2.0,Austin,1,Small


Beautiful! The table creation was a success and we now have the relevant columns in one table. Next, we can join the cost tables to find out the actual costs of each animal and make this altogether another table.

# Joining the new table with the cost tables

In [87]:


CREATE TABLE animal_costs AS
SELECT
    au.animaltype,
    ac.age,
    lc.location,
    sc.size,
    lc.costs size_cost,
    ac.costs ag_cost,
    sc.cost size_cost
    -- We need to add a total cost column
    sc.costs + ac.costs + lc.costs AS total
FROM animals_updated au
JOIN age_costs ac
USING (age)
JOIN location_costs lc
USING (location)
JOIN size_costs sc
ON
    au.size = sc.size
    AND
    au.animaltype = sc.animaltype;

SyntaxError: invalid syntax (<ipython-input-87-af82107ab99c>, line 1)

The query is throwing the error:
    JOIN/USING types double precision and character varying cannot be matched
    
Checking the data types in the animals_updated table

In [ ]:
%%sql

SELECT
    column_name,
    data_type
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    TABLE_NAME = 'animals_updated'

This table has the age data_type as double precision. This needs to be changed to varchar to match the age 
data type in the age_costs table.

In [ ]:
%%sql
ALTER TABLE animals_updated
ALTER COLUMN age TYPE VARCHAR(50);

# Attempting to create new table once again

In [ ]:
%%sql

CREATE TABLE animal_costs AS
SELECT
    au.animaltype,
    ac.age,
    lc.location,
    sc.size,
    lc.costs location_cost,
    ac.costs ag_cost,
    sc.costs size_cost,
    -- We need to add a total cost column
    sc.costs + ac.costs + lc.costs AS total
FROM animals_updated au
JOIN age_costs ac
USING (age)
JOIN location_costs lc
USING (location)
JOIN size_costs sc
ON
    au.size = sc.size
    AND
    au.animaltype = sc.animaltype;

### Inspecting the new table

In [ ]:
%%sql

SELECT 
    *
FROM animal_costs
LIMIT 10;

This is looking wonderful! Only the final step left.

# Creating the final table

### You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.  
Order your query by animal type in alphabetical order, and then size (in order from small to large). Percentage should be expressed as a number rounded to two decimal places (e.g. 50.25% becomes 50.25).

In [ ]:
%%sql
SELECT
    animaltype,
    size,
    SUM(total) AS total,
    ROUND(SUM(total)*100/ (SELECT SUM(total) FROM animal_costs),2) AS percentage
FROM animal_costs
GROUP BY
    size,
    animaltype
ORDER BY
    animaltype,
    size DESC;